<a href="https://colab.research.google.com/github/DShivaram01/Deep_Fake_Detection-Vgg19/blob/main/VGG19_FF%2B%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
from keras.applications import VGG19
from keras.models import Model
from keras.layers import Dense, Flatten

# Load the pre-trained VGG19 model
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Add a new output layer on top of the pre-trained model
x = base_model.output
x = Flatten()(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Define the function to extract frames from the video
def extract_frames(video_path, num_frames=10):
    frames = []
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % (cap.get(cv2.CAP_PROP_FRAME_COUNT) // num_frames) == 0:
            frame = cv2.resize(frame, (224, 224))
            frame = frame / 255.0
            frame = np.expand_dims(frame, axis=0)
            frames.append(frame)
        frame_count += 1
    cap.release()
    frames = np.concatenate(frames, axis=0)
    return frames

# Load the real videos dataset
real_videos_dir = ['/content/drive/MyDrive/FF++/original_sequences/youtube/c23/videos/000.mp4','/content/drive/MyDrive/FF++/original_sequences/youtube/c23/videos/001.mp4','/content/drive/MyDrive/FF++/original_sequences/youtube/c23/videos/002.mp4','/content/drive/MyDrive/FF++/original_sequences/youtube/c23/videos/003.mp4','/content/drive/MyDrive/FF++/original_sequences/youtube/c23/videos/004.mp4','/content/drive/MyDrive/FF++/original_sequences/youtube/c23/videos/005.mp4']
real_labels = np.zeros(6)  # Replace with the actual labels for real videos

real_frames = []
for video_path in real_videos_dir:
    frames = extract_frames(video_path)
    real_frames.append(frames)
real_frames = np.concatenate(real_frames, axis=0)


# Load the fake videos dataset
fake_videos_dir = ['/content/drive/MyDrive/FF++/manipulated_sequences/Deepfakes/c23/videos/000_003.mp4','/content/drive/MyDrive/FF++/manipulated_sequences/Deepfakes/c23/videos/001_870.mp4','/content/drive/MyDrive/FF++/manipulated_sequences/Deepfakes/c23/videos/002_006.mp4','/content/drive/MyDrive/FF++/manipulated_sequences/Deepfakes/c23/videos/003_000.mp4','/content/drive/MyDrive/FF++/manipulated_sequences/Deepfakes/c23/videos/004_982.mp4','/content/drive/MyDrive/FF++/manipulated_sequences/Deepfakes/c23/videos/005_010.mp4']
fake_labels = np.ones(6)  # Replace with the actual labels for fake videos

fake_frames = []
for video_path in fake_videos_dir:
    frames = extract_frames(video_path)
    fake_frames.append(frames)
fake_frames = np.concatenate(fake_frames, axis=0)



In [ ]:
# Combine the real and fake frames and labels
all_frames = np.concatenate((real_frames, fake_frames), axis=0)
all_labels = np.concatenate((real_labels, fake_labels), axis=0)
all_labels = np.repeat(all_labels, all_frames.shape[0])
# Ensure that the number of samples matches
num_samples = len(all_frames)
all_frames = all_frames[:num_samples]
all_labels = all_labels[:num_samples]

# Verify the shapes of all_frames and all_labels
print('all_frames shape:', all_frames.shape)
print('all_labels shape:', all_labels.shape)

# Train the model
model.fit(
    all_frames,
    all_labels,
    batch_size=32,
    epochs=10,
    validation_split=0.2
)


all_frames shape: (130, 224, 224, 3)
all_labels shape: (130,)
Epoch 1/10
4/4 [==============================] - 185s 52s/step - loss: 0.1466 - accuracy: 1.0000 - val_loss: 4.2294e-06 - val_accuracy: 1.0000
Epoch 2/10
4/4 [==============================] - 103s 26s/step - loss: 1.5860e-06 - accuracy: 1.0000 - val_loss: 2.0246e-08 - val_accuracy: 1.0000
Epoch 3/10
4/4 [==============================] - 123s 32s/step - loss: 1.3282e-08 - accuracy: 1.0000 - val_loss: 7.9672e-10 - val_accuracy: 1.0000
Epoch 4/10
4/4 [==============================] - 103s 26s/step - loss: 6.9305e-10 - accuracy: 1.0000 - val_loss: 9.6444e-11 - val_accuracy: 1.0000
Epoch 5/10
4/4 [==============================] - 101s 25s/step - loss: 1.0395e-10 - accuracy: 1.0000 - val_loss: 2.3289e-11 - val_accuracy: 1.0000
Epoch 6/10
4/4 [==============================] - 125s 34s/step - loss: 3.0881e-11 - accuracy: 1.0000 - val_loss: 8.8315e-12 - val_accuracy: 1.0000
Epoch 7/10
4/4 [==============================] - 100s

In [ ]:
model.fit(
    all_frames,
    all_labels,
    batch_size=32,
    epochs=10,
    validation_split=0.2
)
# Evaluate the model on the validation data
score = model.evaluate(all_frames, all_labels)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])

Epoch 1/10


ValueError: ignored

In [ ]:
!pip install efficientnet


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.5 MB/s eta 0:00:00


In [ ]:
print('all_frames shape:', all_frames.shape)

print('all_labels shape:', all_labels.shape)

all_frames shape: (130, 224, 224, 3)
all_labels shape: (130, 6)


In [ ]:
from efficientnet.keras import EfficientNetB6

base_model = EfficientNetB6(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

# Add a new output layer on top of the pre-trained model
x = base_model.output
x = Flatten()(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

all_labels = all_labels.reshape(-1, 1)
model.fit(
    all_frames,
    all_labels,
    batch_size=32,
    epochs=10,
    validation_split=0.2
)



Epoch 1/10
4/4 [==============================] - 94s 19s/step - loss: 1.4289 - accuracy: 0.8365 - val_loss: 2.3770e-12 - val_accuracy: 1.0000
Epoch 2/10
4/4 [==============================] - 56s 14s/step - loss: 1.7738 - accuracy: 0.9423 - val_loss: 2.5210e-13 - val_accuracy: 1.0000
Epoch 3/10
4/4 [==============================] - 53s 13s/step - loss: 0.4709 - accuracy: 0.9712 - val_loss: 7.2799e-12 - val_accuracy: 1.0000
Epoch 4/10
4/4 [==============================] - 56s 13s/step - loss: 1.0079 - accuracy: 0.9135 - val_loss: 1.2072e-13 - val_accuracy: 1.0000
Epoch 5/10
4/4 [==============================] - 54s 14s/step - loss: 0.6138 - accuracy: 0.9519 - val_loss: 3.4136e-17 - val_accuracy: 1.0000
Epoch 6/10
4/4 [==============================] - 54s 14s/step - loss: 0.2758 - accuracy: 0.9615 - val_loss: 8.1656e-19 - val_accuracy: 1.0000
Epoch 7/10
4/4 [==============================] - 55s 14s/step - loss: 0.5381 - accuracy: 0.9423 - val_loss: 1.1217e-17 - val_accuracy: 1.0000

ValueError: ignored

In [ ]:
frames_diff = all_frames.shape[0] - all_labels.shape[0]

# Adjust the size of all_labels if frames_diff is positive
if frames_diff > 0:
    padding = np.zeros((frames_diff, all_labels.shape[1]))  # Create zero-filled padding
    all_labels = np.vstack((all_labels, padding))  # Concatenate the padding to all_labels

print('all_frames shape:', all_frames.shape)
print('all_labels shape:', all_labels.shape)


all_frames shape: (130, 224, 224, 3)
all_labels shape: (780, 1)


In [ ]:
# Evaluate the model on the validation data

score = model.evaluate(all_frames, all_labels)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])

In [ ]:
from keras.applications import ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Add a new output layer on top of the pre-trained model
x = base_model.output
x = Flatten()(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(
    all_frames,
    all_labels,
    batch_size=32,
    epochs=10,
    validation_split=0.2
)

# Evaluate the model on the validation data
score = model.evaluate(all_frames, all_labels)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])


Epoch 1/10
1/1 [==============================] - 8s 8s/step - loss: 1.3482 - accuracy: 0.6667 - val_loss: 1.0559e-09 - val_accuracy: 1.0000
Epoch 2/10
1/1 [==============================] - 2s 2s/step - loss: 27.3604 - accuracy: 0.3333 - val_loss: 3.6601e-06 - val_accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 2s 2s/step - loss: 11.6434 - accuracy: 0.3333 - val_loss: 1.4245 - val_accuracy: 0.6667
Epoch 4/10
1/1 [==============================] - 2s 2s/step - loss: 4.5484e-05 - accuracy: 1.0000 - val_loss: 15.4853 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 3s 3s/step - loss: 3.5917 - accuracy: 0.6667 - val_loss: 21.9085 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 4s 4s/step - loss: 5.6206 - accuracy: 0.6667 - val_loss: 20.4661 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 2s 2s/step - loss: 4.2168 - accuracy: 0.6667 - val_loss: 13.2469 - val_accuracy: 0.0000e+00
Epoch 8

In [ ]:
from keras.applications import VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Add a new output layer on top of the pre-trained model
x = base_model.output
x = Flatten()(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(
    all_frames,
    all_labels,
    batch_size=32,
    epochs=10,
    validation_split=0.2
)

# Evaluate the model on the validation data
score = model.evaluate(all_frames, all_labels)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])


58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/10
1/1 [==============================] - 10s 10s/step - loss: 0.0465 - accuracy: 1.0000 - val_loss: 41.0409 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 7s 7s/step - loss: 17.9400 - accuracy: 0.6667 - val_loss: 9.1854 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 7s 7s/step - loss: 2.0227 - accuracy: 0.6667 - val_loss: 1.2221e-04 - val_accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 6s 6s/step - loss: 5.2493 - accuracy: 0.3333 - val_loss: 6.9211e-04 - val_accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 9s 9s/step - loss: 0.2473 - accuracy: 0.8889 - val_loss: 0.0394 - val_accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 7s 7s/step - loss: 3.3868e-08 - accuracy: 1.0000 - val_loss: 0.5558 - val_accuracy: 0.6667
Epoch 7/10
1/1 [==============================] - 8s 8s/step - loss: 2.4359e-14 - accurac